## Load the meta data into a dataframe

In [1]:
import os
import shutil
import pandas as pd
from pathlib import Path
from tqdm import tqdm
import re
from exif import Image
from PIL import Image

In [2]:
df_labels = pd.read_excel('../data/meta_data.xlsx', sheet_name='AllAnimals')

df_labels

/home/tom-ratsakatika/miniconda3/envs/camera-traps/lib/python3.12/site-packages/openpyxl/worksheet/_read_only.py:81: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
/home/tom-ratsakatika/miniconda3/envs/camera-traps/lib/python3.12/site-packages/openpyxl/worksheet/_read_only.py:81: UserWarning: Conditional Formatting extension is not supported and will be removed
  for idx, row in parser.parse():


,Sort,Session,Category,GMU,TrapSite,Longitude,Latitude,Altitude,StartDate,EndDate,...,Date,Time,RawName,Species,NoAnimals,Sex,Image quality,Problem,Sequence,Comments
0,1,1,Systematic monitoring of lynx,29PiatraCraiuluiBV,Tamasului,25.172013,45.536521,1463,2018-02-15 00:00:00,2018-04-05 00:00:00,...,2018-01-01 00:00:00,na,I__00016,Bear,1,u,medium,partly,6,date unknown
1,2,1,Systematic monitoring of lynx,29PiatraCraiuluiBV,Tamasului,25.172013,45.536521,1463,2018-02-15 00:00:00,2018-04-05 00:00:00,...,2018-01-01 00:00:00,na,I__00026,European hare,1,u,bad,overexposed,2,date unknown
2,3,1,Systematic monitoring of lynx,29PiatraCraiuluiBV,Otetelea,25.184649,45.527437,1515,2018-02-15 00:00:00,2018-04-05 00:00:00,...,2018-01-01 00:00:00,na,I__00002 (2),Fox,1,u,medium,overexposed,1,pooing; date unknown
3,4,1,Systematic monitoring of lynx,29PiatraCraiuluiBV,Otetelea,25.184649,45.527437,1515,2018-02-15 00:00:00,2018-04-05 00:00:00,...,2018-01-01 00:00:00,na,I__00002,Fox,1,u,good,no problem,2,date unknown
4,5,1,Systematic monitoring of lynx,29PiatraCraiuluiBV,Otetelea,25.184649,45.527437,1515,2018-02-15 00:00:00,2018-04-05 00:00:00,...,2018-01-01 00:00:00,na,I__00006,Fox,1,u,bad,overexposed,1,date unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19022,19023,5,Systematic monitoring of feeding points,31BarsaBV,Valea Parului,25.22696,45.584392,880,2021-05-24 00:00:00,2021-06-15 00:00:00,...,2021-06-05 00:00:00,06:06:00,06050011,Squirrel,1,u,medium,no problem,1,na
19023,19024,5,Systematic monitoring of feeding points,31BarsaBV,Valea Parului,25.22696,45.584392,880,2021-05-24 00:00:00,2021-06-15 00:00:00,...,2021-06-07 00:00:00,23:20:00,06070013,Fox,1,u,medium,no problem,1,na
19024,19025,5,Systematic monitoring of feeding points,31BarsaBV,Valea Parului,25.22696,45.584392,880,2021-05-24 00:00:00,2021-06-15 00:00:00,...,2021-06-08 00:00:00,09:03:00,06080014,Roe deer,1,m,medium,no problem,1,na
19025,19026,5,Systematic monitoring of feeding points,31BarsaBV,Valea Parului,25.22696,45.584392,880,2021-06-16 00:00:00,2021-06-16 00:00:00,...,2021-06-16 00:00:00,07:15:00,06160001,Fox,1,u,medium,no problem,2,na


## Save all files with unique filenames in ../data/processed

In [3]:
# Paths configuration
raw_data_path = Path('../data/raw')
processed_data_path = Path('../data/processed')
processed_data_path.mkdir(parents=True, exist_ok=True)

# DataFrame to hold the file information
columns = ['File ID', 'Original Filename', 'File extension'] + [f'Directory {i+1}' for i in range(7)]
df_dirs = pd.DataFrame(columns=columns)

# Collect all files to process for progress tracking
all_files = []
for root, dirs, files in os.walk(raw_data_path):
    for file in files:
        all_files.append((root, file))

# Processing files with progress bar
unique_id = 0  # Unique reference number for each file
for root, file in tqdm(all_files, desc="Copying files"):
    unique_id += 1
    file_path = Path(root) / file
    extension = file.split('.')[-1]
    original_filename = file.rsplit('.', 1)[0]  # filename without the extension
    new_filename = f"{unique_id}.{extension}"
    shutil.copy(file_path, processed_data_path / new_filename)

    # Extract directory path relative to the raw data path
    relative_dir_path = file_path.parent.relative_to(raw_data_path)
    dir_levels = relative_dir_path.parts
    
    # Prepare row for DataFrame
    row = [unique_id, original_filename, extension] + list(dir_levels) + [''] * (7 - len(dir_levels))
    df_dirs.loc[unique_id] = row

##################################

# Function to extract the numerical part
def extract_numerical_prefix(value):
    match = re.match(r'^(\d+(?:_\d+)*)', value)
    return match.group(1) if match else ''

# Filtering df_dirs and explicitly creating a new DataFrame copy
#df_dirs = df_dirs[(df_dirs['Directory 1'] == 'AnimalsOnly_TrainingSubsample') & 
#               (df_dirs['Directory 2'] == 'Season 2 - Animals only')].copy().astype(str)


# Explicitly modifying the DataFrame using .loc
df_dirs.loc[:, 'TrapCode'] = df_dirs['Directory 4'].apply(extract_numerical_prefix)

# Function to determine camera type based on Directory 6
def get_camera_type(value):
    value = value.lower()  # Convert the string to lowercase to make the search case-insensitive
    if 'cudd' in value:
        return 'Cuddeback'
    elif 'bush' in value:
        return 'Bushnell'
    return None  # Return None or an appropriate default value if no match is found

# Apply the function to create the CameraType column
df_dirs['CameraType'] = df_dirs['Directory 6'].apply(get_camera_type)

# Function to extract start and end dates from Directory 5
def extract_date_range(directory_date):
    parts = directory_date.split('_')
    try:
        # Check if the parts are valid dates by trying to convert them to datetime
        pd.to_datetime(parts[0], format='%Y%m%d')  # this will raise an error if not a valid date
        start_date = parts[0]
        end_date = parts[-1]  # takes the last part to handle cases with more than two parts
        return start_date, end_date
    except ValueError:
        # Handle cases where the conversion to date fails (non-date values)
        return "", ""

# Apply the function to create the StartRange and EndRange columns
df_dirs['StartRange'], df_dirs['EndRange'] = zip(*df_dirs['Directory 5'].apply(extract_date_range))


##################################


# Function to extract metadata from an image
def get_image_metadata(file_path):
    try:
        with Image.open(file_path) as img:
            exif_data = img._getexif()
            
            if exif_data is None:
                return None, None
            
            # Extract 'Make' for camera brand and 'DateTimeOriginal' for creation date
            camera_brand = exif_data.get(0x010F)  # 'Make' tag
            created_on = exif_data.get(0x9003)  # 'DateTimeOriginal' tag

            # Convert the 'created_on' to date format if it is not None
            if created_on:
                # Replace ':' with '-' in the date portion of the timestamp
                created_on = created_on.replace(':', '-', 2)
                created_on = pd.to_datetime(created_on).date()

            # Convert camera brand to title case
            if camera_brand:
                camera_brand = camera_brand = str(camera_brand).title()

            return camera_brand, created_on
    except Exception as e:
        # print(f"An error occurred while processing image: {file_path} | Error: {e}")
        return None, None

# Iterate over the DataFrame and update the entries
for index, row in df_dirs.iterrows():
    # Check if any of the CameraType, StartRange, or EndRange is blank
    if pd.isna(row['CameraType']) or pd.isna(row['StartRange']) or pd.isna(row['EndRange']):
        file_name = f"{row['File ID']}.{row['File extension']}"
        file_path = processed_data_path / file_name
        
        # Extract metadata from the file
        camera_brand, created_on = get_image_metadata(file_path)
        
        # Update the DataFrame if metadata is found
        if camera_brand:
            df_dirs.at[index, 'CameraType'] = camera_brand
        if created_on:
            df_dirs.at[index, 'StartRange'] = created_on
            df_dirs.at[index, 'EndRange'] = created_on

# Convert 'StartRange' and 'EndRange' to the same format
df_dirs['StartRange'] = pd.to_datetime(df_dirs['StartRange'], errors='coerce', format='%Y%m%d')
df_dirs['EndRange'] = pd.to_datetime(df_dirs['EndRange'], errors='coerce', format='%Y%m%d')

df_dirs.reset_index(drop=True, inplace=True)

# Save the DataFrame to a CSV file
df_dirs.to_csv('../data/file_index.csv', index=False)
print("Files copied and indexed successfully.")

Copying files: 100%|██████████| 3983/3983 [00:35<00:00, 112.30it/s]


Files copied and indexed successfully.


In [4]:
df_dirs = pd.read_csv('../data/file_index.csv')

## Merge processed file names and metadata tables

In [5]:
df_labels.head()

,Sort,Session,Category,GMU,TrapSite,Longitude,Latitude,Altitude,StartDate,EndDate,...,Date,Time,RawName,Species,NoAnimals,Sex,Image quality,Problem,Sequence,Comments
0,1,1,Systematic monitoring of lynx,29PiatraCraiuluiBV,Tamasului,25.172013,45.536521,1463,2018-02-15 00:00:00,2018-04-05 00:00:00,...,2018-01-01 00:00:00,na,I__00016,Bear,1,u,medium,partly,6,date unknown
1,2,1,Systematic monitoring of lynx,29PiatraCraiuluiBV,Tamasului,25.172013,45.536521,1463,2018-02-15 00:00:00,2018-04-05 00:00:00,...,2018-01-01 00:00:00,na,I__00026,European hare,1,u,bad,overexposed,2,date unknown
2,3,1,Systematic monitoring of lynx,29PiatraCraiuluiBV,Otetelea,25.184649,45.527437,1515,2018-02-15 00:00:00,2018-04-05 00:00:00,...,2018-01-01 00:00:00,na,I__00002 (2),Fox,1,u,medium,overexposed,1,pooing; date unknown
3,4,1,Systematic monitoring of lynx,29PiatraCraiuluiBV,Otetelea,25.184649,45.527437,1515,2018-02-15 00:00:00,2018-04-05 00:00:00,...,2018-01-01 00:00:00,na,I__00002,Fox,1,u,good,no problem,2,date unknown
4,5,1,Systematic monitoring of lynx,29PiatraCraiuluiBV,Otetelea,25.184649,45.527437,1515,2018-02-15 00:00:00,2018-04-05 00:00:00,...,2018-01-01 00:00:00,na,I__00006,Fox,1,u,bad,overexposed,1,date unknown


In [6]:
df_dirs.head()

,File ID,Original Filename,File extension,Directory 1,Directory 2,Directory 3,Directory 4,Directory 5,Directory 6,Directory 7,TrapCode,CameraType,StartRange,EndRange
0,1,am pastrat doar folderele cu poze,txt,WildBoar_VideosAndPictures,WildBoarPhotos,18RaulTarguluiAG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,I_00021a,JPG,WildBoar_VideosAndPictures,WildBoarPhotos,18RaulTarguluiAG,397_E10_Calus,20230920_20231018,CC0485,NaN,397,Cuddeback,2023-10-07,2023-10-07
2,3,I_00021b,JPG,WildBoar_VideosAndPictures,WildBoarPhotos,18RaulTarguluiAG,397_E10_Calus,20230920_20231018,CC0485,NaN,397,Cuddeback,2023-10-07,2023-10-07
3,4,I_00045a,JPG,WildBoar_VideosAndPictures,WildBoarPhotos,18RaulTarguluiAG,397_E10_Calus,20230920_20231018,CC015J,NaN,397,Cuddeback,2020-02-14,2020-02-14
4,5,I_00045b,JPG,WildBoar_VideosAndPictures,WildBoarPhotos,18RaulTarguluiAG,397_E10_Calus,20230920_20231018,CC015J,NaN,397,Cuddeback,2020-02-14,2020-02-14


In [7]:
import pandas as pd
import csv

# Ensure the Date columns are in datetime format and filter out invalid entries
df_labels['Date'] = pd.to_datetime(df_labels['Date'], errors='coerce')
df_dirs['StartRange'] = pd.to_datetime(df_dirs['StartRange'], errors='coerce')
df_dirs['EndRange'] = pd.to_datetime(df_dirs['EndRange'], errors='coerce')

# Dropping NaNs might create a copy, so use 'inplace=True' to do it on the original DataFrame
df_labels.dropna(subset=['Date'], inplace=True)
df_dirs.dropna(subset=['StartRange', 'EndRange'], inplace=True)

# Convert all merge key columns to string to ensure data type consistency
# Use 'loc' to modify the DataFrame in place
df_labels.loc[:, 'GMU'] = df_labels['GMU'].astype(str)
df_labels.loc[:, 'TrapCode'] = df_labels['TrapCode'].astype(str)
df_labels.loc[:, 'Camera'] = df_labels['Camera'].astype(str)
df_labels.loc[:, 'RawName'] = df_labels['RawName'].astype(str)

df_dirs.loc[:, 'Directory 3'] = df_dirs['Directory 3'].astype(str)
df_dirs.loc[:, 'TrapCode'] = df_dirs['TrapCode'].astype(str)
df_dirs.loc[:, 'CameraType'] = df_dirs['CameraType'].astype(str)
df_dirs.loc[:, 'Original Filename'] = df_dirs['Original Filename'].astype(str)

# Perform an inner join based on the specified criteria
potential_matches = pd.merge(df_dirs, df_labels, left_on=['Directory 3', 'TrapCode', 'CameraType', 'Original Filename'], right_on=['GMU', 'TrapCode', 'Camera', 'RawName'], how='inner')

# Filter for valid date ranges
potential_matches = potential_matches[(potential_matches['Date'] >= potential_matches['StartRange']) & (potential_matches['Date'] <= potential_matches['EndRange'])]

# Identify conflicts by counting each Sort ID's occurrences
conflict_counts = potential_matches['Sort'].value_counts()

# Find Sort IDs with more than one occurrence (conflicts)
conflicted_sorts = conflict_counts[conflict_counts > 1].index

# Remove rows associated with conflicted Sort IDs
potential_matches = potential_matches[~potential_matches['Sort'].isin(conflicted_sorts)]

# Save potential matches to CSV
potential_matches.to_csv('../data/potential_matches.csv', index=False, quoting=csv.QUOTE_NONNUMERIC)


In [8]:
import os
import shutil
import pandas as pd


# Define the base paths
source_base_path = '../data/processed'
destination_base_path = '../data/processed'  # Adjust this path as necessary

# Make sure the destination base path exists
os.makedirs(destination_base_path, exist_ok=True)

# Loop through the filtered DataFrame
for index, row in potential_matches.iterrows():
    file_name = f"{row['File ID']}.{row['File extension'].strip()}"
    species_folder = os.path.join(destination_base_path, row['Species'])

    # Create the species directory if it doesn't already exist
    os.makedirs(species_folder, exist_ok=True)

    # Construct full source and destination paths
    source_path = os.path.join(source_base_path, file_name)
    destination_path = os.path.join(species_folder, file_name)

    # Copy the file from source to destination
    if os.path.exists(source_path):  # Check if the file exists to avoid errors
        shutil.copy2(source_path, destination_path)
    else:
        print(f"File not found: {source_path}")

